In [1]:
import requests
import pandas as pd
import numpy as np
import folium

In [2]:
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Downloaded page successfully')
else:
    print('Unable to download. Error code: {}'.format(page.status_code))

Downloaded page successfully


In [3]:
import lxml
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
df_html.dropna(subset=['Borough'], inplace=True)

In [9]:
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [16]:
df_postalcode =pd.DataFrame( df_html.groupby(['Postal code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)]))
df_postalcode.reset_index(inplace=True)
df_postalcode.head(5)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df_html.dropna(subset=['Borough'], inplace=True)

In [20]:
print('The shape of the dataset:',df_postalcode.shape)

The shape of the dataset: (103, 3)


In [21]:
df_postalcode.to_csv('Toronto_data.csv')

In [22]:
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinate = pd.read_csv(url_csv)
df_coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
df_neighborhood = pd.read_csv('Toronto_data.csv',index_col=[0])
df_neighborhood.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [29]:
df_coordinate.rename(columns={'Postal Code': 'Postal code'}, inplace=True)

In [30]:
df_neighborhood_coordinates = pd.merge(df_neighborhood, df_coordinate, on='Postal code')
df_neighborhood_coordinates.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [32]:
df_neighborhood_coordinates.to_csv('Toronto_data2.csv')

In [33]:
df = pd.read_csv('Toronto_data2.csv', index_col=0)
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [34]:
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [35]:
df_torontodata = df[df['Borough'].str.contains('Toronto')]
df_torontodata.reset_index(inplace=True)
df_torontodata.drop('index', axis=1, inplace=True)
df_torontodata.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,The Danforth West / Riverdale,43.679557,-79.352188
2,M4L,East Toronto,India Bazaar / The Beaches West,43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [37]:
print(df_torontodata.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [39]:
boroughs = df_torontodata['Borough'].unique().tolist()

In [41]:
lat_toronto = df_torontodata['Latitude'].mean()
lon_toronto = df_torontodata['Longitude'].mean()
print('The geographical coordinates of Toronto: {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto: 43.66713498717947, -79.38987324871795


In [42]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3))

In [45]:
toronto_map = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_torontodata['Latitude'], 
                                           df_torontodata['Longitude'],
                                           df_torontodata['Borough'], 
                                           df_torontodata['Neighborhood']):
    label_text = borough + ' - ' + neighborhood
    label = folium.Popup(label_text)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(toronto_map)  
    
toronto_map

In [46]:
CLIENT_ID = 'E5YCKOQX2TAQNEINECKNVOG1XO54PULEH0SMCJPV11ROEA30' 
CLIENT_SECRET = 'S1G5JXDQZCTC0NB15WW3244V4VDOOWLCAAGSGX5D4VMZBD20' 
VERSION = '20201704'
LIMIT = 100 
radius = 500 

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
   
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
toronto_venue = getNearbyVenues(names=df_torontodata['Neighborhood'],
                                latitudes=df_torontodata['Latitude'],
                                longitudes=df_torontodata['Longitude'])

The Beaches
The Danforth West / Riverdale
India Bazaar / The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale
St. James Town / Cabbagetown
Church and Wellesley
Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / Design Exchange
Commerce Court / Victoria Hotel
Roselawn
Forest Hill North & West
The Annex / North Midtown / Yorkville
University of Toronto / Harbord
Kensington Market / Chinatown / Grange Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Stn A PO Boxes
First Canadian Place / Underground city
Christie
Dufferin / Dovercourt Village
Little Portugal / Trinity
Brockton / Parkdale Village / Exhibition Place
High Park /

In [50]:
toronto_venue.shape

(1603, 7)

In [51]:
toronto_venue.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
Business reply mail Processing CentrE,14,14,14,14,14,14
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,18,18,18,18,18,18
Central Bay Street,60,60,60,60,60,60
Christie,18,18,18,18,18,18
Church and Wellesley,73,73,73,73,73,73
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,34,34,34,34,34,34


In [53]:
toronto_venue['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood', 'Coffee Shop',
       'Asian Restaurant', 'Greek Restaurant', 'Cosmetics Shop',
       'Italian Restaurant', 'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Pizza Place', 'Bookstore',
       'Restaurant', 'Dessert Shop', 'Juice Bar', 'Bubble Tea Shop',
       'Spa', 'Diner', 'Grocery Store', 'Furniture / Home Store', 'Café',
       'Bakery', 'Caribbean Restaurant', 'Indian Restaurant',
       'Japanese Restaurant', 'American Restaurant', 'Frozen Yogurt Shop',
       'Lounge', 'Gym', 'Fish & Chips Shop', 'Fast Food Restaurant',
       'Sushi Restaurant', 'Liquor Store', 'Park', 'Pet Store',
       'Steakhouse', 'Burrito Place', 'Movie Theater', 'Sandwich Place',
       'Intersection', 'Fish Market', 'Gay Bar', 'Cheese Shop',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Thai Restaurant', 'Seafood Restaurant', 'Stationery Store',
       'Coworking Space', 'Wine Bar', 'Bar',

In [55]:
"Ice Cream Shop" in toronto_venue['Venue Category'].unique()

True

In [58]:
to_onehot = pd.get_dummies(toronto_venue[['Venue Category']], prefix="", prefix_sep="")

to_onehot['Neighborhoods'] = toronto_venue['Neighborhood'] 
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1603, 228)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 228)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,CN Tower / King and Spadina / Railway Lands / ...,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.016667,0.000000,0.000000,0.016667,0.000000,0.016667
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013699,0.000000,0.00,...,0.013699,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027397
7,Commerce Court / Victoria Hotel,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [62]:
len(to_grouped[to_grouped["Ice Cream Shop"] > 0])

12

In [64]:
to_icecream = to_grouped[["Neighborhoods","Ice Cream Shop"]]
to_icecream.head(9)

,Neighborhoods,Ice Cream Shop
0,Berczy Park,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000
2,Business reply mail Processing CentrE,0.000000
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000
4,Central Bay Street,0.033333
5,Christie,0.000000
6,Church and Wellesley,0.000000
7,Commerce Court / Victoria Hotel,0.010000
8,Davisville,0.000000


In [66]:
from sklearn.cluster import KMeans
toclusters = 3

to_clustering = to_icecream.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(to_clustering)

kmeans.labels_[0:20]

array([2, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 1, 2, 2, 0],
      dtype=int32)

In [67]:
to_merged = to_icecream.copy()
to_merged["Cluster Labels"] = kmeans.labels_

In [68]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head(5)

,Neighborhood,Ice Cream Shop,Cluster Labels
0,Berczy Park,0.000000,2
1,Brockton / Parkdale Village / Exhibition Place,0.000000,2
2,Business reply mail Processing CentrE,0.000000,2
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000,2
4,Central Bay Street,0.033333,1


In [69]:
to_merged = to_merged.join(toronto_venue.set_index("Neighborhood"), on="Neighborhood")
print(to_merged.shape)
to_merged.head()

(1603, 9)


,Neighborhood,Ice Cream Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,2,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,2,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,2,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,2,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,2,43.644771,-79.373306,Starbucks,43.644285,-79.369771,Coffee Shop


In [70]:
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged.head()

,Neighborhood,Ice Cream Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,First Canadian Place / Underground city,0.010000,0,43.648429,-79.382280,King Taps,43.648476,-79.382058,Gastropub
25,Richmond / Adelaide / King,0.010638,0,43.650571,-79.384568,Hy's Steakhouse,43.649505,-79.382919,Steakhouse
25,Richmond / Adelaide / King,0.010638,0,43.650571,-79.384568,Cafe Landwer,43.648753,-79.385367,Café
25,Richmond / Adelaide / King,0.010638,0,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel
25,Richmond / Adelaide / King,0.010638,0,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza


In [71]:
map_clusters = folium.Map(location=[lat_toronto, lon_toronto],zoom_start=14)

markers_colors={}
markers_colors[0] = 'red'
markers_colors[1] = 'blue'
markers_colors[2] = 'green'
markers_colors[3] = 'yellow'
markers_colors[4] = 'cyan'
markers_colors[5] = 'black'
for lat, lon, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Cluster Labels']):
    
    
    folium.features.CircleMarker(
        [lat, lon],
        radius=5,
       
        color =markers_colors[cluster],
        fill_color=markers_colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [72]:
to_merged.loc[(to_merged['Cluster Labels'] ==0) & (to_merged['Venue Category'] == 'Ice Cream Shop') ]

,Neighborhood,Ice Cream Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
32,Studio District,0.025000,0,43.659526,-79.340923,Ed's Real Scoop,43.660656,-79.342019,Ice Cream Shop
24,Regent Park / Harbourfront,0.020833,0,43.654260,-79.360636,Greg's Ice Cream,43.650102,-79.360440,Ice Cream Shop
11,First Canadian Place / Underground city,0.010000,0,43.648429,-79.382280,deKEFIR,43.650433,-79.379393,Ice Cream Shop
29,St. James Town,0.012500,0,43.651494,-79.375418,deKEFIR,43.650433,-79.379393,Ice Cream Shop
19,Little Portugal / Trinity,0.023810,0,43.647927,-79.419750,Bang Bang Ice Cream & Bakery,43.646246,-79.419553,Ice Cream Shop
14,Harbourfront East / Union Station / Toronto Is...,0.010000,0,43.640816,-79.381752,Lick It Gelato,43.639256,-79.384650,Ice Cream Shop
25,Richmond / Adelaide / King,0.010638,0,43.650571,-79.384568,deKEFIR,43.650433,-79.379393,Ice Cream Shop
37,Toronto Dominion Centre / Design Exchange,0.010000,0,43.647177,-79.381576,deKEFIR,43.650433,-79.379393,Ice Cream Shop
7,Commerce Court / Victoria Hotel,0.010000,0,43.648198,-79.379817,deKEFIR,43.650433,-79.379393,Ice Cream Shop


In [73]:
to_merged.loc[(to_merged['Cluster Labels'] ==1) & (to_merged['Venue Category'] == 'Ice Cream Shop') ]

,Neighborhood,Ice Cream Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
16,India Bazaar / The Beaches West,0.050000,1,43.668999,-79.315572,Brett's Ice Cream,43.667222,-79.312831,Ice Cream Shop
36,The Danforth West / Riverdale,0.046512,1,43.679557,-79.352188,Dolce Gelato,43.677773,-79.351187,Ice Cream Shop
36,The Danforth West / Riverdale,0.046512,1,43.679557,-79.352188,La Diperie,43.677530,-79.352295,Ice Cream Shop
4,Central Bay Street,0.033333,1,43.657952,-79.387383,Kekou Gelato House,43.656099,-79.392715,Ice Cream Shop
4,Central Bay Street,0.033333,1,43.657952,-79.387383,Arctic Bites,43.656085,-79.392913,Ice Cream Shop


RESULT :
By analyzing nearby venues, we can conclude that the cluster 1 does not have many ice cream shops in that areas.
Thus, it would be suitable to select these locations for opening ice cream shops.
Therefore, locations like Central Bay Street, Riverdale, The Beaches West, Commerce court will be good to open a new Ice Cream Shop.